# Single arm motion planning

This notebook shows how to use the SingleArmOmplPlanner with Drake to plan a collision-free path between two joint configurations.

In [ ]:
from pathlib import Path
import numpy as np
from airo_planner.utils import files
from pydrake.geometry import Meshcat
from pydrake.math import RigidTransform
from pydrake.geometry import MeshcatVisualizer
from pydrake.planning import RobotDiagramBuilder, SceneGraphCollisionChecker
from pydrake.visualization import ApplyVisualizationConfig, VisualizationConfig

## 1. Setting up Drake for collision checking 

### 1.1 Start building the Drake diagram

Building Drake diagram can be quite complex. 
To simplify this Drake has a helper class called `RobotDiagramBuilder` that helps linking all the Drake systems.

Using this class and its attirbutes we can easily add URDFs and weld them together.
As far as I know, a weld is exactly the same as a fixed joint in URDF.

In [ ]:
robot_diagram_builder = RobotDiagramBuilder()  # time_step=0.001 even when I set timestep I get the mimic joint warning
scene_graph = robot_diagram_builder.scene_graph()
plant = robot_diagram_builder.plant()
builder = robot_diagram_builder.builder()
parser = robot_diagram_builder.parser()

# Add visualizer
meshcat = Meshcat()
visualizer = MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat)

# Load URDF files
resources_root = str(files.get_resources_dir())
ur5e_urdf = Path(resources_root) / "robots" / "ur5e" / "ur5e.urdf"
robotiq_2f_85_gripper_urdf = Path(resources_root) / "grippers" / "2f_85_gripper" / "urdf" / "robotiq_2f_85_static.urdf"
table_urdf = "table.urdf"

arm_index = parser.AddModelFromFile(str(ur5e_urdf), model_name="arm_left")
gripper_index = parser.AddModelFromFile(str(robotiq_2f_85_gripper_urdf), model_name="gripper")
table_index = parser.AddModelFromFile(str(table_urdf))

# Weld some frames together
world_frame = plant.world_frame()
arm_frame = plant.GetFrameByName("base_link", arm_index)
arm_wrist_frame = plant.GetFrameByName("wrist_3_link", arm_index)
gripper_frame = plant.GetFrameByName("base_link", gripper_index)
table_frame = plant.GetFrameByName("base_link", table_index)

plant.WeldFrames(world_frame, arm_frame)
plant.WeldFrames(arm_wrist_frame, gripper_frame)
plant.WeldFrames(world_frame, table_frame, RigidTransform([0, 0, 0]))

### 1.2 Finalize the diagram

In [ ]:
# These 4 lines are only for collision visualization
builder = robot_diagram_builder.builder()
plant.Finalize()
config = VisualizationConfig(publish_contacts=True, enable_alpha_sliders=True)
ApplyVisualizationConfig(config, builder=builder, plant=plant, meshcat=meshcat)

# A diagram is needed in the constructor of the SceneGraphCollisionChecker
# However, calling .Build() prevents us from adding more models, e.g. runtime obstacles
diagram = robot_diagram_builder.Build()

# Create default contexts ~= state
context = diagram.CreateDefaultContext()
plant_context = plant.GetMyContextFromRoot(context)
diagram.ForcedPublish(context)

### 1.3 Creating the collision checker

In [ ]:
collision_checker = SceneGraphCollisionChecker(
    model=diagram,
    robot_model_instances=[arm_index],
    edge_step_size=0.125, # Arbitrary value: we don't use the CheckEdgeCollisionFree
    env_collision_padding = 0.005,
    self_collision_padding = 0.005,
)

In [ ]:
start_joints = np.deg2rad([0, -90, -90, -90, 90, 0])

q_all = plant.GetPositions(plant_context)
collision_checker.CheckConfigCollisionFree(q_all)

In [ ]:
plant.SetPositions(plant_context, arm_index, start_joints)
diagram.ForcedPublish(context)

In [ ]:
goal_joints = np.deg2rad([180, -135, 95, -50, -90, -90])
collision_checker.CheckConfigCollisionFree(goal_joints)

## 2. Integration with OMPL

For integration with OMPL, we provide a `SingleArmOmplPlanner`, which handles conversion to OMPL data types and creates OMPL objects. 

The only "external" thing this class requires, a function tells it whether a given joint configuration is valid or not.

In this case, the `CheckConfigCollisionFree` method of the `SceneGraphCollisionChecker` we made can be used for this purpose.

In [ ]:
from cloth_tools.ompl.single_arm_planner import SingleArmOmplPlanner

planner = SingleArmOmplPlanner(is_state_valid_fn=collision_checker.CheckConfigCollisionFree)

In [ ]:
path = planner.plan_to_joint_configuration(start_joints, goal_joints)

if path is None:
    print("No path found")
else:
    with np.printoptions(precision=3, suppress=True):
        print("Amount of waypoints:", len(path))
        print(path[0])
        print(path[-1])


# 3. Visualizing the path in Drake

Because we added a MeshcatVisualizer to the diagram, we can visualize the path in Drake by simply force publishing the waypoints to the Diagram and sleeping in between.

TODO: use the Meshcat support for animations for this instead.

Run the cell below and look at Meshcat in your browser to see your robot moving.

In [ ]:
import time

total_time = 8.0

for joint_configuration in path:
    plant.SetPositions(plant_context, arm_index, joint_configuration)
    diagram.ForcedPublish(context)
    time.sleep(total_time / len(path))